In [11]:
#!/usr/bin/env python3
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [44]:
pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
ale-py                    0.10.1
anyio                     4.5.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
backcall                  0.2.0
beautifulsoup4            4.12.3
bleach                    6.1.0
Brotli                    1.0.9
cached-property           1.5.2
cachetools                5.5.0
certifi                   2024.8.30
cffi                      1.17.0
charset-normalizer        3.4.0
cloudpickle               3.1.0
comm                      0.2.2
debugpy                   1.6.7
decorator                 4.4.2
defusedxml                0.7.1
EasyProcess               1.1
entrypoints               0.4
exceptiongroup            1.2.2
executing                 2.1.0
Farama-Notifications  

In [2]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
print(output)
print(output.size)

tensor([[0.4192, 0.5213, 0.0595],
        [0.6201, 0.1152, 0.2647]])
<built-in method size of Tensor object at 0x792944afcf90>


In [12]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x) 

In [24]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
      print(obs)
      print(obs[0].shape)
      obs_v = torch.FloatTensor(obs[0])
      obs_v_1d = obs_v.unsqueeze(0)
      print(obs_v_1d.shape)
      print(net(obs_v))
      print(type(net(obs_v)))
      print(net(obs_v).shape)
      print(net(obs_v).size())
      #sm(net(obs_v))
      print(sm(net(obs_v_1d))
      act_probs_v = sm(net(obs_v_1d))
      #print(act_probs_v.ndim)
      #print(act_probs_v.shape)
      act_probs = act_probs_v.data.numpy()[0]
      #print(act_probs.ndim)
      #print(act_probs.shape)
      action = np.random.choice(len(act_probs), p=act_probs)
      obs, reward, terminated,truncated, info = env.step(action)
      episode_reward += reward
      episode_steps.append(EpisodeStep(observation=obs, action=action))
        if terminated:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs
        #return print(act_probs_v.ndim , act_probs.ndim )

SyntaxError: invalid syntax (4215724458.py, line 23)

In [5]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [6]:
if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

NameError: name 'iterate_batches' is not defined

In [7]:
import torch
import torchvision.transforms as T

# Create a sample tensor
tensor = torch.randn(3, 224, 224)

# Define the resize transform
resize = T.Resize((128, 128))

# Apply the transform
resized_tensor = resize(tensor)

print(resized_tensor.shape)

import torch
import torch.nn.functional as F

# Create a sample tensor
tensor = torch.randn(3, 224, 224)

# Resize the tensor
resized_tensor = F.interpolate(tensor.unsqueeze(0), size=(128, 128), mode='bilinear', align_corners=False).squeeze(0)

print(resized_tensor.shape)

torch.Size([3, 128, 128])
torch.Size([3, 128, 128])


In [41]:
import torch

yhat = torch.Tensor([[0.5, 1.5, 0.1], [2.2, 1.3, 1.7]])
print(yhat)
# tensor([[0.5000, 1.5000, 0.1000],
#         [2.2000, 1.3000, 1.7000]])

y = torch.Tensor([1, 2])
print(y)
# tensor([1, 2])

loss = torch.nn.CrossEntropyLoss()
cel = loss(input=yhat, target=y)
print(cel)
# tensor(0.8393)

tensor([[0.5000, 1.5000, 0.1000],
        [2.2000, 1.3000, 1.7000]])
tensor([1., 2.])


RuntimeError: expected scalar type Long but found Float

In [43]:
import torch

yhat = torch.Tensor([0.5, 1.5, 0.1])
print(yhat)
# tensor([0.5000, 1.5000, 0.1000])

#y = torch.Tensor([0, 1, 0]).to(torch.long)
y =  = torch.Tensor([0, 1, 0])
print(y)
# tensor([0, 1, 0])

loss = torch.nn.CrossEntropyLoss()
cel = loss(input=yhat, target=y)
print(cel)

SyntaxError: invalid syntax (2573623178.py, line 8)

In [54]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 2)
    
    def forward(self, x):
        return self.linear(x)

# Creating an instance of the model and performing a forward pass
model = SimpleModel()
input_data = torch.randn(1, 10)
output = model(input_data)
print(output)

tensor([[0.0903, 0.1525]], grad_fn=<AddmmBackward0>)


In [68]:
print(output[0])
print(output[0,-1])
output[-1,0]

tensor([0.0903, 0.1525], grad_fn=<SelectBackward0>)
tensor(0.1525, grad_fn=<SelectBackward0>)


tensor(0.0903, grad_fn=<SelectBackward0>)